<a href="https://colab.research.google.com/github/Aydin-ab/CV_DMTet/blob/main/DMTet_Tut_(David).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies: Kaolin and PyTorch3D

In [1]:
from google.colab import drive, auth

# from googleapiclient.discovery import build
# auth.authenticate_user()
# drive_service = build('drive', 'v3')

drive.mount('/content/drive')
from pathlib import Path
INSTALL_PATH = Path("/content/drive/MyDrive/CV_DMTet/")
%cd  $INSTALL_PATH

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CV_DMTet


In [2]:
# reinstall cython, install usd-core (for 3D rendering), and clone into kaolin repo
!pip uninstall Cython --yes
!pip install Cython==0.29.20  --quiet
!pip install usd-core --quiet

Found existing installation: Cython 0.29.20
Uninstalling Cython-0.29.20:
  Successfully uninstalled Cython-0.29.20


In [3]:
# installing kaolin and check version
%env IGNORE_TORCH_VER=1
%env KAOLIN_INSTALL_EXPERIMENTAL=1
KAOLIN_PATH = INSTALL_PATH / "kaolin"
%cd $INSTALL_PATH
!if [ ! -d $KAOLIN_PATH ]; then git clone --recursive https://github.com/NVIDIAGameWorks/kaolin; fi;
%cd $KAOLIN_PATH
SETUP_CHECK = KAOLIN_PATH / "kaolin" / "version.py"
!echo Checking if $SETUP_CHECK exists
!if [ ! -f $SETUP_CHECK ]; then python setup.py develop; fi;
# !python -c "import kaolin; print(kaolin.__version__)"

env: IGNORE_TORCH_VER=1
env: KAOLIN_INSTALL_EXPERIMENTAL=1
/content/drive/MyDrive/CV_DMTet
/content/drive/MyDrive/CV_DMTet/kaolin
Checking if /content/drive/MyDrive/CV_DMTet/kaolin/kaolin/version.py exists


In [4]:
!python setup.py install_lib install_scripts build
# !python setup.py --help

setup.py:36: UserWarning: Kaolin is compatible with PyTorch >=1.6.0, <=1.13.0, but found version 1.13.0+cu116. Continuing with the installed version as IGNORE_TORCH_VER is set.
  warnings.warn(
Compiling kaolin/cython/ops/mesh/triangle_hash.pyx because it depends on /usr/local/lib/python3.8/dist-packages/Cython/Includes/libcpp/vector.pxd.
Compiling kaolin/cython/ops/conversions/mise.pyx because it depends on /usr/local/lib/python3.8/dist-packages/Cython/Includes/libcpp/vector.pxd.
[1/2] Cythonizing kaolin/cython/ops/conversions/mise.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/MyDrive/CV_DMTet/kaolin/kaolin/cython/ops/conversions/mise.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing kaolin/cython/ops/mesh/triangle_hash.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWa

# Import packages

In [5]:
import numpy as np
import torch
import kaolin
import sys
import os

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        print(f"version_str : {version_str}")
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

from kaolin.ops.conversions import (
    trianglemeshes_to_voxelgrids,
    marching_tetrahedra,
    voxelgrids_to_cubic_meshes,
    voxelgrids_to_trianglemeshes,
)

from kaolin.ops.mesh import (
    index_vertices_by_faces
)

from kaolin.io.shapenet import (
    ShapeNetV2
)

from kaolin.metrics.trianglemesh import (
    point_to_mesh_distance,

)

from torch.utils.data import DataLoader

# add path for demo utils functions 
sys.path.append(os.path.abspath(''))
sys.path.append('/content/drive/MyDrive/CV_DMTet/')

# Import Dataset: Subset of ShapeNetV2

In [6]:
import pytorch3d

In [7]:
# state_dict = torch.load('/content/drive/MyDrive/CV_DMTet/shapenet.pvcnn.c1.pth.tar')
# state_dict.keys()

In [8]:
sys.path.append(INSTALL_PATH / "examples")
sys.path.append(INSTALL_PATH / "examples" / "tutorial")

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

In [15]:
# # path to the point cloud to be reconstructed
# pcd_path = KAOLIN_PATH / "examples/samples/bear_pointcloud.usd"
# # path to the output logs (readable with the training visualizer in the omniverse app)
logs_path = '/content/drive/MyDrive/CV_DMTet/Logs'

# # We initialize the timelapse that will store USD for the visualization apps
timelapse = kaolin.visualize.Timelapse(logs_path)

#Load Tetrahedral grid

DMTet starts from a uniform tetrahedral grid of predefined resolution, and uses a network to predict the SDF value as well as deviation vector at each grid vertex.

Here we load the pre-generated tetrahedral grid using Quartet at resolution 128, which has roughly the same number of vertices as a voxel grid of resolution 65. We use a simple MLP + positional encoding to predict the SDF and deviation vectors in DMTet, and initialize the encoded SDF to represent a sphere.

In [16]:
# Uniform Tetrahedral Grid
tets_verts = torch.tensor(np.load(KAOLIN_PATH / "examples/samples/128_verts.npz")['data'], dtype=torch.float, device=device)
tets = torch.tensor(([np.load(KAOLIN_PATH / 'examples/samples/128_tets_{}.npz'.format(i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)
print(tets_verts, tets)


tensor([[ 0.5000,  0.5000,  0.4844],
        [ 0.4844,  0.5000,  0.4922],
        [ 0.4922,  0.4844,  0.4844],
        ...,
        [-0.1719, -0.5000,  0.4766],
        [-0.1562, -0.5000,  0.4688],
        [-0.1562, -0.4922,  0.4688]], device='cuda:0') tensor([[     0,      1,      2,      3],
        [     2,      3,      1,      4],
        [     5,      3,      0,      2],
        ...,
        [277409, 272920, 272914, 272919],
        [272919, 277409, 272920, 274866],
        [277409, 277400, 272920, 274866]], device='cuda:0')


# Loading from ShapeNet

In [17]:
# Not using for now, using libigl tutorial x-cylinder below
SHAPENET_PATH = "/content/drive/MyDrive/CV_DMTet/Core"
#SHAPENET_PATH = "/content/drive/MyDrive/FALL 2022/Computer Vision/Project/Core"
SYNSETS_IDS = ['02747177', '02801938', '02808440', '02818832', '02828884', '02871439', '02876657', '02880940', '02924116', '02933112']
shapenet_train = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True)
shapenet_test = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True, train=False)

In [18]:
mesh = shapenet_train[10]['mesh'] # change the index here for different models
gt_verts = mesh[0].to(device)
gt_faces = mesh[1].to(device)
print(gt_verts.shape)
timelapse.add_mesh_batch(
    category='gt',
    vertices_list=[gt_verts.cpu()],
    faces_list=[gt_faces.cpu()]
)
gt_sample = int(0.99*gt_verts.shape[0])

torch.Size([6038, 3])


In [19]:
# Clone sample meshes from libigl tutorial 1x
LIBIGL_TUTORIAL_DATA = INSTALL_PATH / "libigl-tutorial-data"
print(f"LIBIGL_TUTORIAL_DATA: {LIBIGL_TUTORIAL_DATA}")
%cd $INSTALL_PATH
!if [ ! -d $LIBIGL_TUTORIAL_DATA ]; then git clone --recursive https://github.com/libigl/libigl-tutorial-data.git; fi;
%cd $KAOLIN_PATH

LIBIGL_TUTORIAL_DATA: /content/drive/MyDrive/CV_DMTet/libigl-tutorial-data
/content/drive/MyDrive/CV_DMTet
/content/drive/MyDrive/CV_DMTet/kaolin


In [20]:
# Load cylinder (along x-axis) mesh (V,F) = (42,80)
mesh = kaolin.io.obj.import_mesh(LIBIGL_TUTORIAL_DATA / "arm.obj") #xcylinder.obj
gt_verts = mesh[0].to(device)
gt_faces = mesh[1].to(device)
timelapse.add_mesh_batch(
    category='gt',
    vertices_list=[gt_verts.cpu()],
    faces_list=[gt_faces.cpu()]
)
gt_sample = int(0.99*gt_verts.shape[0])

In [21]:
points = kaolin.ops.mesh.sample_points(gt_verts.unsqueeze(0), gt_faces, gt_sample)[0][0]
center = (points.max(0)[0] + points.min(0)[0]) / 2
max_l = (points.max(0)[0] - points.min(0)[0]).max()
points = ((points - center) / max_l)* 0.9
points.to(device)
timelapse.add_pointcloud_batch(category='input',
                               pointcloud_list=[points.cpu()], points_type = "usd_geom_points")

# Convert model to watertight meshes

We used a voxelization with resolution of 64 to predict the sdf and extract surface. 

In [22]:
# wt_grid
# Convert mesh (V,F) to a voxel grid 
# voxels = kaolin.ops.conversions.trianglemeshes_to_voxelgrids(
#     vertices=mesh_verts.unsqueeze(0).to(device),
#     faces=mesh_faces.to(device),
#     resolution=16
# )

# # Convert the voxel grid back into a mesh as GroundTruth
# # I am not sure this step is necessary
# # The intention is to have mesh_vertices, mesh_faces for the voxel grid for loss function
# wt_verts, wt_faces = kaolin.ops.conversions.voxelgrids_to_cubic_meshes(voxels)
# wt_verts, wt_faces = wt_verts[0], wt_faces[0]

# max_gt_len = (mesh_verts.max(0)[0] - mesh_faces.min(0)[0]).max()
# max_vox_len = (wt_verts.max(0)[0] - wt_verts.min(0)[0]).max()
# scale = max_gt_len / max_vox_len
# center = (wt_verts.max(0)[0] + wt_verts.min(0)[0]) / 2
# wt_verts = ((wt_verts - center) * scale)

# timelapse.add_mesh_batch(
#     category='watertight_test',
#     vertices_list=[wt_verts.cpu()],
#     faces_list=[wt_faces.cpu()]
# )

In [23]:
# Using Kaolin bear point cloud data (PCD) right now since tutorial uses PCD
# pcd_path = KAOLIN_PATH / "examples/samples/bear_pointcloud.usd"
# points = kaolin.io.usd.import_pointclouds(str(pcd_path))[0].points.to(device)
# if points.shape[0] > 100000:
#     idx = list(range(points.shape[0]))
#     np.random.shuffle(idx)
#     idx = torch.tensor(idx[:100000], device=points.device, dtype=torch.long)    
#     points = points[idx]

# # The reconstructed object needs to be slightly smaller than the grid to get watertight surface after MT.
# # The idea is that we want our point cloud to expand since it will converge faster if all face expansions are outward.
# center = (points.max(0)[0] + points.min(0)[0]) / 2
# max_l = (points.max(0)[0] - points.min(0)[0]).max()
# points = ((points - center) / max_l)* 0.9
# timelapse.add_pointcloud_batch(category='input',
#                                pointcloud_list=[points.cpu()], points_type = "usd_geom_points")

kaolin.ops.conversions.voxelgrids_to_cubic_meshes(

Convert voxelgrids to meshes by replacing each occupied voxel with a cuboid mesh (unit cube). Each cube has 8 vertices and 6 (for quadmesh) or 12 faces (for triangular mesh). Internal faces are ignored. If is_trimesh==True, this function performs the same operation as “Cubify” defined in the ICCV 2019 paper “Mesh R-CNN”: https://arxiv.org/abs/1906.02739.

Parameters
voxelgrids (torch.Tensor) – binary voxel array, of shape .

is_trimesh (optional, bool) – if True, the outputs are triangular meshes. Otherwise quadmeshes are returned. Default: True.

Returns
The list of vertices for each mesh.

The list of faces for each mesh.

Return type
(list[torch.Tensor], list[torch.LongTensor])

kaolin.ops.conversions.marching_tetrahedra(vertices, tets, sdf, return_tet_idx=False)¶
Convert discrete signed distance fields encoded on tetrahedral grids to triangle meshes using marching tetrahedra algorithm as described in An efficient method of triangulating equi-valued surfaces by using tetrahedral cells. The output surface is differentiable with respect to input vertex positions and the SDF values. For more details and example usage in learning, see Deep Marching Tetrahedra: a Hybrid Representation for High-Resolution 3D Shape Synthesis NeurIPS 2021.


# SDF model

We follow the paper recommandation and use a four-layer
MLPs with hidden dimensions 256, 256, 128 and 64, respectively

In [24]:
# Since we skip PVCNN, input dimension is just the coordinates of each grid

SDF_MLP_CONFIG = {
    'input_dim' : 3, # Coordinates of the grid's vertices
    'hidden_dims' : [256, 256, 128, 64],
    'output_dim' : 1, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
    'multires': 2
}

lr = 0.001
laplacian_weight = 0.1
iterations = 2000
save_every = 100
multires = 2
grid_res = 128

In [143]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

# Comes from the DMTet tutorial 
# MLP + Positional Encoding
class Decoder(torch.nn.Module):
    def __init__(self, input_dims = 3, internal_dims = 128, output_dims = 4, hidden = 5, multires = 2):
        super().__init__()
        self.embed_fn = None
        if multires > 0:
            embed_fn, input_ch = get_embedder(multires)
            self.embed_fn = embed_fn
            input_dims = input_ch

        net = (torch.nn.Linear(input_dims, internal_dims, bias=False), torch.nn.ReLU())
        for i in range(hidden-1):
            net = net + (torch.nn.Linear(internal_dims, internal_dims, bias=False), torch.nn.ReLU())
        net = net + (torch.nn.Linear(internal_dims, output_dims, bias=False),)
        self.net = torch.nn.Sequential(*net)

    def forward(self, p):
        if self.embed_fn is not None:
            p = self.embed_fn(p)
        out = self.net(p)
        return out

    def pre_train_sphere(self, iter):
        print ("Initialize SDF to sphere")
        loss_fn = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(list(self.parameters()), lr=1e-4)
        

        for i in tqdm(range(iter)):
            p = torch.rand((1024,3), device='cuda') - 0.5
            ref_value  = torch.sqrt((p**2).sum(-1)) - 0.3
            output = self(p)
            loss = loss_fn(output[...,0], ref_value)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Pre-trained MLP", loss.item())

# Positional Encoding from https://github.com/yenchenlin/nerf-pytorch/blob/1f064835d2cca26e4df2d7d130daa39a8cee1795/run_nerf_helpers.py
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()
        
    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d
            
        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']
        
        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)
            
        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d
                    
        self.embed_fns = embed_fns
        self.out_dim = out_dim
        
    def embed(self, inputs):
        return torch.cat([fn(inputs) for fn in self.embed_fns], -1)

def get_embedder(multires):
    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 3,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }
    
    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

In [144]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [145]:
from tqdm import tqdm
# # Initialize model and create optimizer
sdf_model = Decoder(multires=2).to(device)
sdf_model.pre_train_sphere(1000)
# print(sdf_model)
# print('\n\n')
# summary(sdf_model, input_size=tets_verts.shape)

Initialize SDF to sphere


100%|██████████| 1000/1000 [00:02<00:00, 457.21it/s]

Pre-trained MLP 4.892159267910756e-06


In [136]:
# sdf_model = MLP(SDF_MLP_CONFIG).to(device)
# print(sdf_model)
# print('\n\n')
# summary(sdf_model, input_size=tets_verts.shape)

Little Test:

In [29]:
# signed distance fields encoded on tetrahedral grid

# pred_sdfs dim = (tets_vertices.shape[0])
# f_vs ie f_v feature vector

# sdf, delta x_i, delta y_i, delta z_i, 
# sdf = binary occupancy of tetrahedron 
pred = sdf_model(tets_verts)
print(f'pred shape is : {pred.shape}')

# pred_sdfs, f_vs = sdf_model(tets_verts)

print(f'Input grid shape is : {tets_verts.shape}')
# print(f'Output shape of the predicted SDFs should be {tets_verts.shape[0], 1} and it actually is {tuple(pred_sdfs.shape)}')
# print(f'Output shape of the feature vectors f_vs should be {tets_verts.shape[0], 64} and it actually is {tuple(f_vs.shape)}')
# print(pred_sdfs)

pred shape is : torch.Size([277410, 4])
Input grid shape is : torch.Size([277410, 3])


# Set up Optimizer

In [30]:
sdf_vars = [p for _, p in sdf_model.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

# Training

In [31]:
# # takes in a module and applies the specified weight initialization
# def weights_init_normal(m):
#     '''Takes in a module and initializes all linear layers with weight
#         values taken from a normal distribution.'''

#     classname = m.__class__.__name__
#     # for every Linear layer in a model
#     if classname.find('Linear') != -1:
#         y = m.in_features
#     # m.weight.data shoud be taken from a normal distribution
#         m.weight.data.normal_(0.0,1/np.sqrt(y))
#     # m.bias.data should be 0
#         m.bias.data.fill_(0)

In [32]:
# sdf_model.apply(weights_init_normal)

In [83]:
# Laplacian regularization using umbrella operator (Fujiwara / Desbrun).
# https://mgarland.org/class/geom04/material/smoothing.pdf
from pytorch3d import loss
def laplace_regularizer_const(pred_mesh_verts, pred_mesh_faces):
    term = torch.zeros_like(pred_mesh_verts)
    norm = torch.zeros_like(pred_mesh_verts[..., 0:1])

    v0 = pred_mesh_verts[pred_mesh_faces[:, 0], :]
    v1 = pred_mesh_verts[pred_mesh_faces[:, 1], :]
    v2 = pred_mesh_verts[pred_mesh_faces[:, 2], :]

    term.scatter_add_(0, pred_mesh_faces[:, 0:1].repeat(1,3), (v1 - v0) + (v2 - v0))
    term.scatter_add_(0, pred_mesh_faces[:, 1:2].repeat(1,3), (v0 - v1) + (v2 - v1))
    term.scatter_add_(0, pred_mesh_faces[:, 2:3].repeat(1,3), (v0 - v2) + (v1 - v2))

    two = torch.ones_like(v0) * 2.0
    norm.scatter_add_(0, pred_mesh_faces[:, 0:1], two)
    norm.scatter_add_(0, pred_mesh_faces[:, 1:2], two)
    norm.scatter_add_(0, pred_mesh_faces[:, 2:3], two)

    term = term / torch.clamp(norm, min=1.0)

    return torch.mean(term**2)

def loss_f(pred_mesh_verts, pred_mesh_faces, pts, it):
    # sample_number = mesh_verts.shape[0] * .5
    pred_points = kaolin.ops.mesh.sample_points(pred_mesh_verts.unsqueeze(0), pred_mesh_faces, 50000)[0][0]
    
    chamfer = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), pts.unsqueeze(0)).mean()
    # chamfer = loss.chamfer_distance(pred_points.unsqueeze(0), pts.unsqueeze(0))[0]
    # print(chamfer)
    # chamfer = torch.tensor(chamfer, requires_grad=True)
    # chamfer.clone().detach().requires_grad_(True)
    if it > iterations//2:
        lap = laplace_regularizer_const(pred_mesh_verts, pred_mesh_faces)
        return chamfer + lap * laplacian_weight
    return chamfer 


def sdf_train(iterations, model, optimizer, scheduler):
  # Set to training mode
  model.train()

  for it in range(iterations):
      pred = model(tets_verts)
      
      # Replace pred_sdfs with the new SDF
      # Replace deform with surface refinement 

      pred_sdfs, deform = pred[:,0], pred[:,1:]

      
      verts_deformed = tets_verts + torch.tanh(deform) / grid_res # constraint deformation to avoid flipping tets
      # print(f"verts_deformed : {verts_deformed.shape}")

      pred_mesh_verts, pred_mesh_faces = marching_tetrahedra(verts_deformed.unsqueeze(0), tets, pred_sdfs.unsqueeze(0))
      pred_mesh_verts, pred_mesh_faces = pred_mesh_verts[0], pred_mesh_faces[0]
      # print(f"pred_mesh_verts empty: {pred_mesh_verts.nelement() == 0}")
      # print(f"pred_sdfs : {pred_sdfs}")
      # print(f"verts_deformed : {verts_deformed}")
      # print(f"tets : {tets}")
      
      loss = loss_f(pred_mesh_verts, pred_mesh_faces, points, it)
      # print(loss.shape)
      # loss = F.mse_loss(pred_sdfs, gt_sdfs.squeeze(0).unsqueeze(1), reduction='sum')

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      scheduler.step()

      if (it) % save_every == 0 or it == (iterations - 1): 
          print ('Iteration {} - loss: {}'.format(it, loss))
          # save reconstructed mesh
          timelapse.add_mesh_batch(
              iteration=it+1,
              category='extracted_mesh',
              vertices_list=[pred_mesh_verts.cpu()],
              faces_list=[pred_mesh_faces.cpu()]
          )


In [90]:
sdf_model = Decoder(multires=2).to(device)
sdf_model.pre_train_sphere(1000)
sdf_vars = [p for _, p in sdf_model.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time
# ~12 min. Speed up?
sdf_train(iterations, sdf_model, sdf_optimizer, sdf_scheduler)

Initialize SDF to sphere


100%|██████████| 1000/1000 [00:02<00:00, 448.14it/s]


Pre-trained MLP 5.996178970235633e-06
Iteration 0 - loss: 0.056564778089523315
Iteration 100 - loss: 0.00035423709778115153
Iteration 200 - loss: 0.0001338023430434987
Iteration 300 - loss: 7.230698247440159e-05
Iteration 400 - loss: 6.221563671715558e-05
Iteration 500 - loss: 4.6070210373727605e-05
Iteration 600 - loss: 4.5048262109048665e-05
Iteration 700 - loss: 3.970160105382092e-05
Iteration 800 - loss: 3.556008232408203e-05
Iteration 900 - loss: 0.0007746580522507429
Iteration 1000 - loss: 0.00108240672852844
Iteration 1100 - loss: 0.00017891003517434
Iteration 1200 - loss: 6.203664815984666e-05
Iteration 1300 - loss: 0.00010509687854209915
Iteration 1400 - loss: 5.496963785844855e-05
Iteration 1500 - loss: 4.85259442939423e-05
Iteration 1600 - loss: 4.529577199718915e-05
Iteration 1700 - loss: 4.188844468444586e-05
Iteration 1800 - loss: 3.8608737668255344e-05
Iteration 1900 - loss: 3.71672649635002e-05
Iteration 1999 - loss: 0.00017360292258672416


 # Surface refinement utils

In [99]:
# Get edges lists of shape [E, 2] from face list of shape [V, 4]
def get_edges(input):
  c = torch.combinations(torch.arange(input.size(1)), r=2)
  x = input[:,None].expand(-1,len(c),-1).cpu()
  idx = c[None].expand(len(x), -1, -1)
  x = x.gather(dim=2, index=idx)

  return x.view(-1, *x.shape[2:])


# Extract tets under certain sdf restrictions:
# if eps = 0, return all surface tetrahedrons
# if eps > 0, return all tetrahedrons whose vertices' sdfs are all in the range [-thresh, thresh]

# return all tetrahedrons whose 4 vertices i = [0,1,2,3] have abs(sdf(v[i])) <= eps

def extract_surf_tets(tets, sdf, eps=0, non_surf=False):

  assert eps >= 0

  if eps == 0:
    mask = (sdf[tets] > 0)
    mask_int = mask.long()
    t = mask_int.sum(1)
    surf_tets = tets[(t > 0) & (t < 4)]
  else:
    mask = (abs(sdf[tets]) <= eps)
    mask_int = mask.long()
    t = mask_int.sum(1)
    surf_tets = tets[t == 4]

  surf_tets_tuple = surf_tets.unique(return_inverse=True)
  surf_tets_idx, surf_tets = surf_tets_tuple[0], surf_tets_tuple[1]

  if non_surf:
    non_surf_tets = tets[t == 0]
    non_surf_tets_tuple = non_surf_tets.unique(return_inverse=True)
    non_surf_tets_idx, non_surf_tets = non_surf_tets_tuple[0], non_surf_tets_tuple[1]
    return surf_tets_idx, surf_tets, non_surf_tets_idx, non_surf_tets

  return surf_tets_idx, surf_tets

#Get output from initial MLP

def get_pred_sdfs(model, tets_verts):
  model.eval()

  with torch.no_grad():
    pred_sdfs, f_vs = model(tets_verts)

  return pred_sdfs, f_vs

# Get ground truth sdf from input verts and faces
# input shape: [batch_size, num_vertices, 3], [num_faces, 4], [batch_size, num_points, 3]
# output shape: [num_points, 1]

def get_gt_sdfs(gt_verts, gt_faces, points, f=None):

  if f == None:
    f = index_vertices_by_faces(gt_verts, gt_faces)
  sdf,_,_ = point_to_mesh_distance(points, f)

  s = kaolin.ops.mesh.check_sign(gt_verts, gt_faces, points)
  sdf = sdf[s == True]

  return sdf.squeeze(0).unsqueeze(1)

In [100]:
pred = sdf_model(tets_verts)
pred_sdfs = pred[:,0]
# print(pred_sdfs.shape)
# mask = (torch.abs(pred_sdfs) <= 3e-1)
# masked_tets = mask[tets].long()
# masked_tets.sum(1)

In [101]:
# pred_sdfs[abs(pred_sdfs) <= 1e-3]

In [102]:
# surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, pred_sdfs, eps=0)
# surf_tets_verts = tets_verts[surf_tets_verts_idx]
surf_tets_verts_idx, surf_tets_faces = extract_surf_tets(tets, pred_sdfs, eps=0)
surf_tets_verts = tets_verts[surf_tets_verts_idx]
    
timelapse.add_mesh_batch(
            category='tet_surface',
            vertices_list=[surf_tets_verts.cpu()],
            faces_list=[surf_tets_faces.cpu()]
        )

# Surface refinement model

In [103]:
"""
Graph-res net:
Identify surface tetrahedral, build adj matrix, 
"""

from tqdm import tqdm

from torch import nn
from torch.nn import functional as F
from pytorch3d.ops import GraphConv

# a single res block layer with dimension 256 & 128
class GResBlock(nn.Module): 
    def __init__(self, in_dim, hidden_dim, activation=None):
        super(GResBlock, self).__init__()

        self.conv1 = GraphConv(in_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, in_dim)
        self.activation = F.relu if activation else None
    
    def forward(self, inputs):
        input, adj = inputs[0], inputs[1]
        x = self.conv1(input, adj)
        if self.activation:
          x = self.activation(x)
        x = self.conv2(x, adj)
        if self.activation:
          x = self.activation(input + x)
        
        return [x, adj]

class GBottleneck(nn.Module):
    def __init__(self, block_num, in_dim, hidden_dim, out_dim, activation=None):
        super(GBottleneck, self).__init__()

        resblock_layers = [GResBlock(in_dim=hidden_dim[0], hidden_dim=hidden_dim[1], activation=activation)
                          for _ in range(block_num)]
        self.blocks = nn.Sequential(*resblock_layers)
        self.conv1 = GraphConv(in_dim, hidden_dim[0])

        self.activation = F.relu if activation else None
    
    def forward(self, inputs, adj):
        x = self.conv1(inputs, adj)
        if self.activation:
          x = self.activation(x)
        x = self.blocks([x, adj])[0]
        if self.activation:
          x = self.activation(x)

        return x

class GCN_Res(nn.Module):
    def __init__(self, config):
        super(GCN_Res, self).__init__()

        self.in_dim = config['in_dim']
        self.hidden_dim = config['hidden_dim']
        self.out_dim = config['out_dim']
        self.activation = config['activation']
        self.mlp_hdim = config['mlp_hdim']
        self.mlp_odim = config['mlp_odim']

        self.gcn_res = nn.ModuleList([GBottleneck(2, self.in_dim, self.hidden_dim, self.out_dim, self.activation)])

        self.sdf_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 1, bias=False),
        )

        self.deform_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 3, bias=False),
        )

        self.feature_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], self.mlp_hdim[1], bias=False),
        )
        
    def forward(self, inputs, adj):

        x = self.gcn_res[0](inputs, adj)

        sdf = self.sdf_mlp(x)
        deform = self.deform_mlp(x)
        feature = self.feature_mlp(x)
        
        return sdf, deform, feature


In [104]:
lambda_cd = 500
lambda_norm = 1e-6
lambda_def = 1
lamdba_sdf = 0.4

def gcn_loss(iterations, pred_verts, pred_faces, gt_verts, gt_faces, it):

    #surface alignment loss
    m = pytorch3d.structures.Meshes([pred_verts, gt_verts], [pred_faces, gt_faces])
  
    # pc = pytorch3d.ops.sample_points_from_meshes(m, num_samples=50000)
    pred_points = kaolin.ops.mesh.sample_points(pred_verts.unsqueeze(0), pred_faces, 50000)[0][0]
    gt_points = kaolin.ops.mesh.sample_points(gt_verts.unsqueeze(0), gt_faces, 50000)[0][0]

    # pred_points,gt_points = pc[0], pc[1]
    Lcd = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), gt_points.unsqueeze(0)).mean()
    # chamfer.clone().detach().requires_grad_(True)
    
    Lnorm = pytorch3d.loss.mesh_normal_consistency(m)
    if it > iterations//2:
      lap = pytorch3d.loss.mesh_laplacian_smoothing(m)
      return lambda_cd*Lcd + lap * laplacian_weight + lambda_norm*Lnorm # + sdf_loss + deform_loss
    return lambda_cd*Lcd + lambda_norm*Lnorm #+ sdf_loss + deform_loss

# Train GCN Model for Surface Refinement

In [105]:
CONFIG_GCNRES = {
    'in_dim': 68,
    'hidden_dim': [128, 256],
    'out_dim': 128,
    'activation': True,
    'mlp_hdim': [128,64],
    'mlp_odim': 68,
}

# Same set of Hyperparam is applied
lr = 1e-4
laplacian_weight = 0.1
gcn_iterations = 5000
save_every = 100
multires = 2
grid_res = 128

In [127]:
p = torch.rand((2,3), device='cuda') - 0.5 # -.5 to .5
print(p)
ref_value  = torch.sqrt((p**2).sum(-1)) - .3
ref_value

tensor([[-0.4506,  0.3199,  0.0129],
        [-0.0751,  0.2240, -0.0084]], device='cuda:0')


tensor([ 0.2528, -0.0636], device='cuda:0')

In [128]:
output = sdf_model(p)
print(output)
print(output[...,0]) #gets the 1st column

tensor([[ 0.4844, -0.1064,  0.6909, -0.4993],
        [ 0.2055, -0.0546,  0.2990, -0.2611]], device='cuda:0',
       grad_fn=<MmBackward0>)
tensor([0.4844, 0.2055], device='cuda:0', grad_fn=<SelectBackward0>)


In [205]:
from tqdm import tqdm

def pre_train_sphere(self, iter):
  print ("Initialize SDF to sphere")
  loss_fn = torch.nn.MSELoss()
  optimizer = torch.optim.Adam(list(self.parameters()), lr=1e-4)
  

  for i in tqdm(range(iter)):
      p = torch.rand((1024,3), device='cuda') - 0.5
      ref_value  = torch.sqrt((p**2).sum(-1)) - 0.3
      output = self(p)
      loss = loss_fn(output[...,0], ref_value)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  print("Pre-trained MLP", loss.item())

# Spheres centered at (0,0,0) with radius=1
# https://math.stackexchange.com/questions/1784106/how-do-i-compute-the-closest-points-on-a-sphere-given-a-point-outside-the-sphere
# dist = sdf = sqrt(a^2 + b^2 + c^2)
# Why do we subtract 0.3 though??
# dir = deform = (- a/dist, -b/dist, -c/dist)
def gcn_pretrain(iterations, gcn_model, pred_sdf, tets_verts, tets, thresh):
  gcn_model.train()

  assert thresh >= 0
  
  loss_fn = torch.nn.MSELoss()
  optimizer = torch.optim.Adam(list(gcn_model.parameters()), lr=1e-4)
  
  # Random feature vector
  tets_verts_features = torch.zeros((tets_verts.shape[0],64), device=device)

  # p = torch.rand((pred_sdf.shape[0],3), device='cuda')*2 - 0.5
  ref_sdf  = torch.sqrt((tets_verts**2).sum(-1)) - 0.3 #sdf
  ref_deform = - tets_verts / ref_sdf.unsqueeze(1) #deform

  surf_tets_verts_idx, surf_tets_faces = extract_surf_tets(tets, pred_sdf, thresh)
  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])
  print(f"surf_tets_verts_idx : {surf_tets_verts_idx}")

  # Correcting for surface
  ref_sdf = ref_sdf[surf_tets_verts_idx]
  ref_deform = ref_deform[surf_tets_verts_idx]

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = pred_sdf[surf_tets_verts_idx].detach()
  surf_tets_edges = torch.clone(get_edges(surf_tets_faces).to(device))
  print(f"surf_tets_edges : {surf_tets_edges.shape}")
  
  for it in tqdm(range(iterations)):
    verts_f = torch.cat((surf_tets_verts, surf_sdfs.unsqueeze(1), surf_tets_verts_features), dim=1)

    sdf, deform, fv = gcn_model(verts_f, surf_tets_edges)

    update_verts = surf_tets_verts + torch.tanh(deform) / grid_res
    update_sdfs = surf_sdfs + sdf
    # print(update_sdfs.shape)
    # print(ref_sdf.shape)

    # loss = F.mse_loss(update_sdfs, ref_sdf, reduction='mean')
    loss = loss_fn(update_sdfs, ref_sdf)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("Pre-trained GCN", loss.item())


In [206]:
refine_model = GCN_Res(CONFIG_GCNRES).to(device)
pred = sdf_model(tets_verts)
pred_sdf = pred[:,0]
gcn_pretrain(1000, refine_model, pred_sdf, tets_verts, tets, 0)

surf_tets_verts_idx : tensor([ 64469,  64472,  64477,  ..., 242053, 242305, 242313], device='cuda:0')
surf_tets_edges : torch.Size([168174, 2])
torch.Size([9933, 3])
torch.Size([9933, 1])
torch.Size([9933, 1, 64])


100%|██████████| 1000/1000 [00:30<00:00, 32.95it/s]


Pre-trained GCN 4.0547922253608704e-05


In [193]:
"""
Identify surface Tetrahedral
input: f = [v, s, F_vol, f_64]
output: f = [delta v, delta s, f_64]

update v = v + delta v, s = s + delta s
generate new meshes by marching tet from v and s; calculate loss

loss = chamfer + sdf_loss + deform_loss
"""
from pytorch3d import loss

def gcn_train(iterations, gcn_model, optimizer, scheduler, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  """

  Firstly, based on predicted sdf value, extract out all surface tetrahedrons
  i.e. 1-3 vertices of tetrahedrons are with different sdf signs
  return value: 
  --> surf_tets_idx: origin index of vertices as in tet grid
  --> surf_tets_faces: reindexed faces list start from 0

  """

  surf_tets_verts_idx, surf_tets_faces, non_surf_idx, non_surf_tets = extract_surf_tets(tets, pred_sdf, thresh, True)
  surf_tets_edges = get_edges(surf_tets_faces).to(device)

  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  # obtain per vertex features: 
  # sdf, position, feature vector

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = pred_sdf[surf_tets_verts_idx].detach()
  surf_verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

  non_surf_verts = torch.clone(tets_verts[non_surf_idx])

  # calculate ground truth sdf value over all vertices in tet mesh

  # gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces).to(device)
  # gt_sdfs, _, _ = point_to_mesh_distance(tets_verts.unsqueeze(0), gt_f)
  # gt_sdfs = gt_sdfs.squeeze(0).unsqueeze(1)

  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)

  gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)
  
  gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, tets_verts.unsqueeze(0), gt_f)

  for it in range(iterations):
    
    """
    
    Secondly, predict with a 2-layer GCN followed by 2-layer MLP
    output:
    --> [delta v, delta s, new f_s] \in R^68

    """

    sdf, deform, fv = gcn_model(surf_verts_f, surf_tets_edges)
    
    """

    Update surface position, sdf, and f_s

    """

    #updated sdf

    us = sdf.detach()
    update_sdfs = torch.clone(pred_sdf)
    update_sdfs[surf_tets_verts_idx] += us

    #update vertices positions

    vd = torch.tanh(deform).detach()
    update_tets_verts = torch.clone(tets_verts)
    update_tets_verts[surf_tets_verts_idx] += vd / grid_res

    #update vertices features

    vfvs = fv.detach()
    update_tets_f = torch.clone(tets_verts_features)
    update_tets_f[surf_tets_verts_idx] += vfvs

    """

    Marching Tetrahedra based on new sdf value and deformed vertices in the tet grid

    """

    mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(update_tets_verts.unsqueeze(0), tets, update_sdfs.squeeze(1).unsqueeze(0))
    mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

    """

    Compute Loss for First surface refinement: 
    Normal consistency + surface alignment + laplacian smooth + sdf L2-reg + deform L2-reg

    """

    # L2 sdf reg: 

    # Calculate SDF non_surface tetrahedron in newly generated mesh

    p_sdfs = get_gt_sdfs(mesh_verts.unsqueeze(0), mesh_faces, non_surf_verts.unsqueeze(0))

    # Get ground truth SDF value of non_surface tetrahedron vertices and truncate at +- 0.3 to focus on surface

    ns_g = gt_sdfs[non_surf_idx]
    mask = (ns_g >= -0.3) & (ns_g <= 0.3)
    p = p_sdfs[mask]
    g = ns_g[mask]

    # Also calculate surface sdf loss

    s_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_tets_verts[surf_tets_verts_idx].unsqueeze(0), gt_f)

    sdf_loss = F.mse_loss(sdf, s_sdfs - surf_sdfs, reduction='sum') #+F.mse_loss(p, g, reduction='sum') +

    #L2 deform reg

    deform_loss = F.mse_loss(deform, torch.zeros(deform.shape).to(device), reduction='sum')

    #surface alignment loss

    r_loss = gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it)

    loss = r_loss + 0.4*sdf_loss + deform_loss

    optimizer.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    loss.backward(retain_graph=True)
    optimizer.step()
    scheduler.step()

    if (it) % save_every == 0 or it == (iterations - 1): 
      print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, loss, mesh_verts.shape[0], mesh_faces.shape[0]))
      
      # save reconstructed mesh
      timelapse.add_mesh_batch(
          iteration=it+1,
          category='extracted_mesh',
          vertices_list=[mesh_verts.cpu()],
          faces_list=[mesh_faces.cpu()]
      )

# GCN Train with Subdivision

In [ ]:
# from pytorch3d import loss

# def gcn_train_wsubdiv(iterations, gcn_model, optimizer, scheduler, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
#   gcn_model.train()

#   assert thresh >= 0

#   """

#   Firstly, based on predicted sdf value, extract out all surface tetrahedrons
#   i.e. 1-3 vertices of tetrahedrons are with different sdf signs
#   return value: 
#   --> surf_tets_idx: origin index of vertices as in tet grid
#   --> surf_tets_faces: reindexed faces list start from 0

#   """

#   # Get surface and non-surface tetrahedron indices

#   surf_tets_verts_idx, surf_tets_faces, non_surf_idx, non_surf_tets = extract_tet(tets, pred_sdf, thresh, True)

#   # Use surface faces to obtain edge list

#   surf_tets_edges = get_edges(surf_tets_faces).to(device)


#   # Use surface indices obtain per vertex features: 

#   # Surface vertices positions:

#   surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

#   # Surface per vertex feature vector:

#   surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])

#   # Surface per vertex predicted sdf value:

#   surf_sdfs = torch.clone(pred_sdf[surf_tets_verts_idx])

#   # Cat all features together as the input into model

#   surf_verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

#   # Obtain non_surface vertices position

#   non_surf_verts = torch.clone(tets_verts[non_surf_idx])

#   # Calculate ground truth sdf value over all vertices in tet mesh

#   gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)
#   gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)

#   gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, tets_verts.unsqueeze(0), gt_f)

#   for it in range(100):
    
#     """
    
#     Secondly, predict with a 2-layer GCN followed by 2-layer MLP
#     output:
#     --> [delta v, delta s, new f_s] \in R^68

#     """

#     sdf, deform, fv = gcn_model(surf_verts_f, surf_tets_edges)
    
#     """

#     Update surface position, sdf, and f_s

#     """

#     #updated sdf

#     us = sdf.detach()
#     update_sdfs = torch.clone(pred_sdf)
#     update_sdfs[surf_tets_verts_idx] += us

#     #update vertices positions

#     vd = torch.tanh(deform).detach()
#     update_tets_verts = torch.clone(tets_verts)
#     update_tets_verts[surf_tets_verts_idx] += vd / grid_res

#     #update vertices features

#     vfvs = fv.detach()
#     update_tets_f = torch.clone(tets_verts_features)
#     update_tets_f[surf_tets_verts_idx] += vfvs

#     """

#     Volume subdivision:
#     Re-identify surface tets based on updated sdf value and subdivide identified surface tets


#     """

#     # Extract surface tetrahedrons based on updated sdf value

#     d_surf_tets_verts_idx, d_surf_tets_faces = extract_tet(tets, update_sdfs, thresh)
#     if d_surf_tets_faces.shape[0] == 0:
#       d_surf_tets_verts_idx, d_surf_tets_faces = surf_tets_verts_idx, surf_tets_faces

#     # Use new surface indices to get new surface vertices positions

#     d_surf_tets_verts = torch.clone(update_tets_verts[d_surf_tets_verts_idx])

#     # Use new surface indices to get new surface per vertex sdf and features

#     d_surf_tets_verts_features = torch.clone(update_tets_f[d_surf_tets_verts_idx])
#     d_surf_sdfs = torch.clone(update_sdfs[d_surf_tets_verts_idx])

#     # Cat new surface per vertex features together for subdivide

#     d_surf_verts_f = torch.cat((d_surf_sdfs, d_surf_tets_verts_features), dim=1)

#     # Perform subdivide, returns subdivided surface tetrahedron 

#     d_tets_verts, d_tets_faces, d_tets_fvs = kaolin.ops.mesh.subdivide_tetmesh(d_surf_tets_verts.unsqueeze(0), d_surf_tets_faces, d_surf_verts_f.unsqueeze(0))
#     d_tets_verts, d_tets_fvs = d_tets_verts[0], d_tets_fvs[0]
#     d_tets_sdfs = d_tets_fvs[:, 0]


#     # Get surface edges from tetrahedron faces

#     d_tets_edges = get_edges(d_tets_faces).to(device)

#     # Cat to obtain feature input into GCN

#     d_tets_fvs = torch.cat((d_tets_verts, d_tets_fvs), dim=1)

#     d_sdf, d_deform, d_fv = gcn_model(d_tets_fvs, d_tets_edges)

#     # Updated sdf

#     d_update_sdfs = d_tets_sdfs.unsqueeze(1).detach() + d_sdf #since we need this to back propagate


#     # Update vertices positions

#     d_vd = torch.tanh(d_deform).detach()
#     d_update_tets_verts = d_tets_verts + d_vd / grid_res

#     # Update vertices features

#     d_vfvs = d_fv.detach()
#     d_update_tets_f = d_tets_fvs[:, 4:] + d_vfvs

#     """

#     Marching Tetrahedra based on new sdf value and deformed vertices in the tet grid

#     """

#     mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(d_update_tets_verts.unsqueeze(0), d_tets_faces, d_update_sdfs.squeeze(1).unsqueeze(0))
#     mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

#     print(mesh_verts.shape, mesh_faces.shape)

#     """

#     Compute Loss for First surface refinement: 
#     Normal consistency + surface alignment + laplacian smooth + sdf L2-reg + deform L2-reg

#     """

#     # L2 sdf reg: 

#     # Calculate SDF non_surface tetrahedron in newly generated mesh

#     p_sdfs = get_gt_sdfs(mesh_verts.unsqueeze(0), mesh_faces, non_surf_verts.unsqueeze(0))

#     # Get ground truth SDF value of non_surface tetrahedron vertices and truncate at +- 0.3 to focus on surface

#     ns_g = gt_sdfs[non_surf_idx]
#     mask = (ns_g >= -0.3) & (ns_g <= 0.3)
#     p = p_sdfs[mask]
#     g = ns_g[mask]

#     # Also calculate surface sdf loss

#     # s_sdfs, _, _ = point_to_mesh_distance(update_tets_verts[surf_tets_verts_idx].unsqueeze(0), gt_f)
#     # s_sign = kaolin.ops.mesh.check_sign(gt_verts.unsqueeze(0), gt_faces, update_tets_verts[surf_tets_verts_idx].unsqueeze(0))
#     # s_sdfs[s_sign==False] *= -1

#     # s_sdfs = s_sdfs.squeeze(0).unsqueeze(1)

#     s_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, d_update_tets_verts.unsqueeze(0), gt_f)

#     sdf_loss = F.mse_loss(p, g, reduction='sum') + F.mse_loss(d_update_sdfs, s_sdfs, reduction='sum')

#     #L2 deform reg

#     deform_loss = F.mse_loss(d_deform, torch.zeros(d_deform.shape).to(device), reduction='sum')

#     #surface alignment loss

#     r_loss = gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it)
    
#     loss = r_loss + deform_loss + 0.4*sdf_loss 

#     optimizer.zero_grad()
#     torch.autograd.set_detect_anomaly(True)
#     loss.backward(retain_graph=True)
#     optimizer.step()
#     scheduler.step()

#     # print('Iteration {} - loss: {}'.format(it, loss))

#     if (it) % save_every == 0 or it == (iterations - 1): 
#       print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, loss, mesh_verts.shape[0], mesh_faces.shape[0]))
      
#       # save reconstructed mesh
#       timelapse.add_mesh_batch(
#           iteration=it+1,
#           category='extracted_mesh',
#           vertices_list=[mesh_verts.cpu()],
#           faces_list=[mesh_faces.cpu()]
#       )

# Run Surface Refinement

In [ ]:
refine_model = GCN_Res(CONFIG_GCNRES).to(device)

print(refine_model)

pre_vars = [p for _, p in refine_model.named_parameters()]
pre_optimizer = torch.optim.Adam(pre_vars, lr=lr)

In [ ]:
# pred_sdfs, _ = get_pred_sdfs(sdf_model, tets_verts)


pred = sdf_model(tets_verts)
pred_sdfs, deform = pred[:,0], pred[:,1:]
print(extract_tets(tets, pred_sdfs, 0.001)[0].shape)

In [ ]:
gcn_pretrain(500, refine_model, pre_optimizer, pred_sdfs, tets_verts, tets, deform, sample_verts, sample_faces, 0.001)

In [ ]:
refine_vars = [p for _, p in refine_model.named_parameters()]
refine_optimizer = torch.optim.Adam(pre_vars, lr=lr)
refine_scheduler = torch.optim.lr_scheduler.LambdaLR(pre_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

In [ ]:
gcn_train(gcn_iterations, refine_model, refine_optimizer, refine_scheduler, pred_sdfs, tets_verts, tets, deform, sample_verts, sample_faces, 0.001)

In [ ]:
# test_gcn = GCN_Res(CONFIG_GCNRES).to(device)

# t_v = [p for _, p in test_gcn.named_parameters()]
# t_op = torch.optim.Adam(t_v, lr=lr)
# t_sch = torch.optim.lr_scheduler.LambdaLR(t_op, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

In [ ]:
# gcn_pretrain(700, test_gcn, t_op, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0)

In [ ]:
# gcn_train_wsubdiv(gcn_iterations, test_gcn, t_op, t_sch, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0)

In [ ]:
"""
to-dos: volume subdivision

identify T_surf's neighbors: i.e. share a same edge

subdivide T_surf to perform another surface refinement
unsubdivded tet, i.e. not surface's neighbor, is dropped to save memory and computation

"""

#neightbor if share an edge
#identify edge based on surf_faces and adj matrix

"""
convert face_lists = [v1, v2, v3, v4] --> 
[
  [v1, v2, v3, E, o1],
  [v2, v3, v4, E, o2],
  [v1, v2, v4, E, o3],
  [v1, v3, v4, E, o4]
]

by torch combinations, E = tet index, o_i = face_i index
"""
def convert_face_lists(tets):
  face_idx = torch.tensor([1,2,3,4])
  tet_face_list = []
  for idx, tet in enumerate(tets):
    tet_idx = torch.full(4, idx)
    tet_faces = torch.combinations(tet, r=3)
    tet_faces = torch.cat((tet_faces, tet_idx, face_idx), dim=1)
    tet_face_list.append(tet_faces)

  tet_face_list = torch.stack(tet_face_list, dim=0)

  def compare(face_1, face_2):
    o1 = face_1[0]-face_2[0]
    o2 = face_1[1]-face_2[1]
    o3 = face_1[2]-face_2[2]

    if o1 <= 0 or (o1 == 0 and o2 < 0) or (o1 == 0 and o2 == 0 and o3 < 0):
      return -1
    elif o1 == 0 and o2 == 0 and o3 == 0:
      return 0
    else: 
      return 1

  sorted(tet_face_list, cmp=compare)
  return tet_face_list

def get_neighbor(surf_tet_verts, surf_tets, tet_verts, tets):
  tet_face_list = convert_face_lists(tets)
  

surf_idx, surf = extract_tet(tets, pred_sdfs, 0.003)

print(surf_idx)

In [ ]:
def get_faces(input):
  c = torch.combinations(torch.arange(input.size(1)), r=3)
  x = input[:,None].expand(-1,len(c),-1).cpu()
  idx = c[None].expand(len(x), -1, -1)
  x = x.gather(dim=2, index=idx)

  return x.view(-1, *x.shape[2:])

In [ ]:
# a = get_faces(tets)
# b = get_faces(tets[surf_idx])

# # OPTIMIZE TF OUT OF THIS
# for f in b:
#   mask = a == f


# Visualization

In [43]:
#Use pyngrok to access localhost:80 on Colab

!pip install pyngrok --quiet 
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2Hzzzh94FgOXssVkSP5Yffz8uYg_By2RMDZLTPx1aXakhYfH"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [87]:
#generating a public url mapped to localhost 80
public_url = ngrok.connect(port=80, proto="http", options={"bind_tls": True, "local": True})
print("Tracking URL:", public_url)

PyngrokNgrokHTTPError: ignored

In [88]:
#Start Kaolin Dash3D on localhost:80 
!kaolin-dash3d --logdir=/content/drive/MyDrive/CV_DMTet/Logs --port=80

Dash3D server starting. Go to: http://localhost:80
2022-12-12 17:31:18,305|    INFO|kaolin.visualize.timelapse| No checkpoints found for type voxelgrid: no files matched pattern voxelgrid*.usd in /content/drive/MyDrive/CV_DMTet/Logs
Traceback (most recent call last):
  File "/usr/local/bin/kaolin-dash3d", line 6, in <module>
    run_main()
  File "/usr/local/lib/python3.8/dist-packages/kaolin/experimental/dash3d/run.py", line 96, in run_main
    server.listen(args.port)
  File "/usr/local/lib/python3.8/dist-packages/tornado/web.py", line 2116, in listen
    server.listen(port, address)
  File "/usr/local/lib/python3.8/dist-packages/tornado/tcpserver.py", line 151, in listen
    sockets = bind_sockets(port, address=address)
  File "/usr/local/lib/python3.8/dist-packages/tornado/netutil.py", line 174, in bind_sockets
    sock.bind(sockaddr)
OSError: [Errno 98] Address already in use
